#Project Team2

Members:
- Martim Teixeira
- Parisa Eimanzadeh
- YuHsuan Wu
- Anupama Bhat
- Sai Sreekar Nelakonda

In [183]:
pip install --upgrade snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import the Snowflake connector library
import snowflake.connector

# Establish a connection to the Snowflake database
# The `connect` method takes several parameters to authenticate and specify the Snowflake account to connect to
conn = snowflake.connector.connect(
    user='MATEIXEIRA',
    password='',
    account='TIB91679'
)


In [23]:
# Create a new cursor object using the `cursor` method from the established Snowflake connection
# The cursor object allows you to execute SQL queries and fetch results
cs = conn.cursor()


In [ ]:
# A SQL command to create a new Snowflake warehouse named "Project_Team2"
cs.execute("CREATE WAREHOUSE IF NOT EXISTS Project_Team2")

In [ ]:
# A SQL command to create a new Snowflake database named "Project_Team2"
cs.execute("CREATE DATABASE IF NOT EXISTS Project_Team2")


In [ ]:
# Switch to the "PROJECT_TEAM2" database
cs.execute("USE PROJECT_TEAM2;")

# Create a new schema named "Work" if it doesn't already exist
cs.execute("CREATE SCHEMA IF NOT EXISTS Work")


Exercise 1

In [35]:

# Switch to the "PROJECT_TEAM2.WORK" schema
cs.execute("USE PROJECT_TEAM2.WORK;")

# Create or replace the "purchases" table with specified columns and data types
cs.execute(
    """
    CREATE OR REPLACE TABLE purchases (
        PurchaseOrderID INTEGER,
        SupplierID INTEGER,
        OrderDate DATE,
        DeliveryMethodID INTEGER,
        ContactPersonID INTEGER,
        ExpectedDeliveryDate DATE,
        SupplierReference STRING,
        IsOrderFinalized BOOLEAN,
        Comments STRING,
        InternalComments STRING,
        LastEditedBy INTEGER,
        LastEditedWhen STRING,
        PurchaseOrderLineID INTEGER,
        StockItemID INTEGER,
        OrderedOuters INTEGER,
        Description STRING,
        ReceivedOuters INTEGER,
        PackageTypeID FLOAT,
        ExpectedUnitPricePerOuter FLOAT,
        LastReceiptDate STRING,
        IsOrderLineFinalized BOOLEAN,
        Right_LastEditedBy INTEGER,
        Right_LastEditedWhen STRING
);
"""
)

# Switch to the "PROJECT_TEAM2.WORK" schema again (redundant but safe)
cs.execute("USE PROJECT_TEAM2.WORK;")

# Create or replace a stage named "purchase_stg" with CSV file format
cs.execute("CREATE OR REPLACE STAGE purchase_stg FILE_FORMAT = (TYPE = 'CSV')")

# Import the glob library to find all CSV files in a directory
import glob

# Get a list of all CSV files in the specified directory
csv_files = glob.glob("./Case_Data/Monthly_PO_Data/Monthly_PO_Data/*.csv")

# Define the Snowflake stage to upload files to
stage = 'PROJECT_TEAM2.WORK.PURCHASE_STG'

# Loop through each CSV file to upload it to the Snowflake stage
for file_path in csv_files:
    print(f"Uploading {file_path}")

    # Upload file to the Snowflake stage
    cs.execute(f"PUT file://{file_path} @{stage}")

# Copy the data from the stage to the "purchases" table
cs.execute(f"COPY INTO PURCHASES FROM @{stage} FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER = 1) ON_ERROR = 'CONTINUE'")

# Print completion message
print(f"Completed uploading {file_path}")


# Drop the "COMMENTS" and "INTERNALCOMMENTS" columns from the "purchases" table
cs.execute("ALTER TABLE PURCHASES DROP COLUMN COMMENTS,INTERNALCOMMENTS;")

Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-1.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-10.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-11.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-12.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-2.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-3.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-4.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-5.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-6.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-7.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-8.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2013-9.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2014-1.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2014-10.csv
Uploading ./Case_Data/Monthly_PO_Data/Monthly_PO_Data/2014-11.csv
Uploading ./Case_Dat

In [110]:
cs.execute("SELECT * FROM PURCHASES LIMIT 10")

Exercise 2

In [111]:
print(cs.fetchmany(2))


[(2025, 4, datetime.date(2016, 5, 2), 7, 2, datetime.date(2016, 5, 22), '293092', True, 'NULL', 'NULL', 10, datetime.datetime(2016, 5, 3, 7, 0), 8158, 77, 3265, '"The Gu" red shirt XML tag t-shirt (White) XXS', 3265, 6, Decimal('84.00'), datetime.date(2016, 5, 3), True, 10, datetime.datetime(2016, 5, 3, 7, 0)), (2025, 4, datetime.date(2016, 5, 2), 7, 2, datetime.date(2016, 5, 22), '293092', True, 'NULL', 'NULL', 10, datetime.datetime(2016, 5, 3, 7, 0), 8159, 78, 3599, '"The Gu" red shirt XML tag t-shirt (White) XS', 3599, 6, Decimal('84.00'), datetime.date(2016, 5, 3), True, 10, datetime.datetime(2016, 5, 3, 7, 0))]


In [213]:
cs.execute("SELECT * FROM PURCHASES LIMIT 10")
print(cs.fetchmany(2))

[(204, 4, datetime.date(2013, 5, 1), 7, 2, datetime.date(2013, 5, 21), '293092', True, 'NULL', 'NULL', 10, datetime.datetime(2013, 5, 2, 7, 0), 832, 77, 263, '"""The Gu"" red shirt XML tag t-shirt (White) XXS"', 263, 6, Decimal('84.00'), datetime.date(2013, 5, 2), True, 10, datetime.datetime(2013, 5, 2, 7, 0)), (204, 4, datetime.date(2013, 5, 1), 7, 2, datetime.date(2013, 5, 21), '293092', True, 'NULL', 'NULL', 10, datetime.datetime(2013, 5, 2, 7, 0), 833, 78, 322, '"""The Gu"" red shirt XML tag t-shirt (White) XS"', 322, 6, Decimal('84.00'), datetime.date(2013, 5, 2), True, 10, datetime.datetime(2013, 5, 2, 7, 0))]


In [16]:
# Add a new column named "POAmount" with data type INT to the "purchases" table
cs.execute("ALTER TABLE PURCHASES ADD COLUMN POAmount INT;")

In [17]:
# Update the "POAmount" column in the "purchases" table with the sum of (ReceivedOuters * ExpectedUnitPricePerOuter)
# for each unique "PurchaseOrderId"

cs.execute("""
UPDATE PURCHASES p1
SET POAmount = (
    SELECT SUM(p2.ReceivedOuters * p2.ExpectedUnitPricePerOuter)
    FROM PURCHASES p2
    WHERE p2.PurchaseOrderId = p1.PurchaseOrderId
)
WHERE EXISTS (
    SELECT 1
    FROM PURCHASES p2
    WHERE p2.PurchaseOrderId = p1.PurchaseOrderId
);
""")

Exercise 3

In [9]:
cs.execute("USE PROJECT_TEAM2.WORK;")
# Create a new stage for supplier invoices
cs.execute("CREATE STAGE IF NOT EXISTS invoice_stg")

# Upload the supplier invoice XML file to the supplier invoices stage
cs.execute("""
    PUT 'file://./Case_Data/Supplier_Transactions_XML.xml' 
@invoice_stg auto_compress=false

""")

# Create a table with one column of type variant
cs.execute("""
    CREATE OR REPLACE TABLE supplier_invoices_var (
        xml_variant VARIANT
    )
""")

# Create a file format for the supplier transactions XML file
cs.execute("""
    CREATE OR REPLACE FILE FORMAT supplier_transactions_xml_ff 
    TYPE = 'XML' 
    STRIP_OUTER_ELEMENT=TRUE
""")

# Copy the supplier transactions XML file into the variant table
cs.execute("""
    COPY INTO supplier_invoices_var 
    FROM @invoice_stg 
    FILE_FORMAT = (FORMAT_NAME = 'supplier_transactions_xml_ff') 
    ON_ERROR = 'CONTINUE' 
    FORCE = TRUE
""")

# Create supplier invoices table with data from the variant table
cs.execute("""
    CREATE OR REPLACE TABLE SUPPLIER_INVOICE AS 
    SELECT 
        REGEXP_SUBSTR(xml_variant::STRING, '<AmountExcludingTax>([^<]+)</AmountExcludingTax>', 1, 1, 'e') AS AmountExcludingTax,
        REGEXP_SUBSTR(xml_variant::STRING, '<PurchaseOrderID>([^<]+)</PurchaseOrderID>', 1, 1, 'e') AS PurchaseOrderID,
        REGEXP_SUBSTR(xml_variant::STRING, '<SupplierID>([^<]+)</SupplierID>', 1, 1, 'e') AS SupplierID,
        REGEXP_SUBSTR(xml_variant::STRING, '<SupplierTransactionID>([^<]+)</SupplierTransactionID>', 1, 1, 'e') AS SupplierTransactionID,
        REGEXP_SUBSTR(xml_variant::STRING, '<TransactionTypeID>([^<]+)</TransactionTypeID>', 1, 1, 'e') AS TransactionTypeID,
        REGEXP_SUBSTR(xml_variant::STRING, '<PaymentMethodID>([^<]+)</PaymentMethodID>', 1, 1, 'e') AS PaymentMethodID,
        REGEXP_SUBSTR(xml_variant::STRING, '<SupplierInvoiceNumber>([^<]+)</SupplierInvoiceNumber>', 1, 1, 'e') AS SupplierInvoiceNumber,
        REGEXP_SUBSTR(xml_variant::STRING, '<TransactionDate>([^<]+)</TransactionDate>', 1, 1, 'e') AS TransactionDate,
        REGEXP_SUBSTR(xml_variant::STRING, '<TaxAmount>([^<]+)</TaxAmount>', 1, 1, 'e') AS TaxAmount,
        REGEXP_SUBSTR(xml_variant::STRING, '<TransactionAmount>([^<]+)</TransactionAmount>', 1, 1, 'e') AS TransactionAmount,
        REGEXP_SUBSTR(xml_variant::STRING, '<OutstandingBalance>([^<]+)</OutstandingBalance>', 1, 1, 'e') AS OutstandingBalance,
        REGEXP_SUBSTR(xml_variant::STRING, '<FinalizationDate>([^<]+)</FinalizationDate>', 1, 1, 'e') AS FinalizationDate,
        REGEXP_SUBSTR(xml_variant::STRING, '<IsFinalized>([^<]+)</IsFinalized>', 1, 1, 'e') AS IsFinalized,
        REGEXP_SUBSTR(xml_variant::STRING, '<LastEditedBy>([^<]+)</LastEditedBy>', 1, 1, 'e') AS LastEditedBy,
        REGEXP_SUBSTR(xml_variant::STRING, '<LastEditedWhen>([^<]+)</LastEditedWhen>', 1, 1, 'e') AS LastEditedWhen
    FROM supplier_invoices_var
""")

Exercise 4

In [11]:
cs.execute("USE PROJECT_TEAM2.WORK;")


# Create or replace a view named "PurchasesAndInvoices" that joins the "PURCHASES" and "SUPPLIER_INVOICE" tables
# The view includes columns from both tables where the "PurchaseOrderID" matches
cs.execute("""
CREATE OR REPLACE VIEW PurchasesAndInvoices AS (
    SELECT A.PurchaseOrderID, A.SupplierID, A.OrderDate, B.AmountExcludingTax, A.POAmount, B.TaxAmount, B.TransactionAmount
    FROM PURCHASES A
    INNER JOIN SUPPLIER_INVOICE B
    ON A.PurchaseOrderID = B.PurchaseOrderID
);
""")


EXERCISE 5

In [106]:

# Create or replace a view named "purchase_orders_and_invoices" that joins the "PURCHASES" and "SUPPLIER_INVOICE" tables
# The view includes columns from both tables where the "PurchaseOrderID" matches
# Additionally, it adds 5 years to the "OrderDate" and calculates the difference between "AmountExcludingTax" and "POAmount"


cs.execute("""
CREATE OR REPLACE VIEW purchase_orders_and_invoices AS (
    SELECT A.PurchaseOrderID, A.SupplierID, dateadd(year, +5, A.OrderDate) AS OrderDate, B.AmountExcludingTax - A.POAmount AS invoiced_vs_quoted, B.TaxAmount, B.TransactionAmount
    FROM PURCHASES A
    INNER JOIN SUPPLIER_INVOICE B
    ON A.PurchaseOrderID = B.PurchaseOrderID
);
""")

Exercise 6

In [19]:
# Import the required libraries: csv for handling CSV files and psycopg2 for PostgreSQL database operations
import csv
import psycopg2

# Define a function to export data from a PostgreSQL table to a CSV file
def export_from_postgres_to_csv():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        dbname='WestCoastImporters',
        user='jovyan',
        password='postgres',
        host='127.0.0.1',
        port=8765
    )
    # Create a cursor object to interact with the database
    cursor = conn.cursor()
    
    # Open a new CSV file and write the data from the PostgreSQL table into it
    with open('supplier_case.csv', 'w', newline='') as f:
        cursor.copy_expert("COPY supplier_case TO STDOUT WITH CSV HEADER;", f)
    
    # Close the cursor and the database connection
    cursor.close()
    conn.close()

# Define a function to generate field definitions based on the first row of a CSV file
def generate_field_definitions(csv_file_path):
    # Open the CSV file in read mode
    with open(csv_file_path, mode ='r') as file:
        # Create a DictReader object to read the CSV file
        csv_reader = csv.DictReader(file)
        
        # Get the field names (column names) from the CSV file
        field_names = csv_reader.fieldnames
        
        # Read the first row to determine the data types of the fields
        first_row = next(csv_reader)
        
        # Initialize an empty list to hold field definitions
        field_definitions = []
        
        # Loop through each field to determine its data type based on the first row
        for field in field_names:
            if first_row[field].isdigit():
                field_type = 'INTEGER'
            else:
                field_type = 'STRING'
            
            # Append the field definition to the list
            field_definitions.append(f"{field} {field_type}")
        
        # Return the field definitions as a comma-separated string
        return ', '.join(field_definitions)


def upload_to_snowflake(): 
    # Create internal stage
    cs.execute("CREATE OR REPLACE STAGE supplier_stg FILE_FORMAT = (TYPE = 'CSV');")
    
    # Upload CSV to internal stage
    cs.execute(f"PUT file://./supplier_case.csv @supplier_stg;")
    
    # Generate CREATE TABLE statement
    field_definitions = generate_field_definitions("supplier_case.csv")
    create_table_query = f"CREATE TABLE supplier_case ({field_definitions});"
    
    # Create the table
    cs.execute(create_table_query)
    
    # Copy data from internal stage to the table
    cs.execute("COPY INTO supplier_case FROM @supplier_stg FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER = 1) ON_ERROR = 'CONTINUE'")
    
    cs.close()
    conn.close()

if __name__ == "__main__":
    export_from_postgres_to_csv()
    upload_to_snowflake()


Exercise 7

In [29]:
cs.execute("USE PROJECT_TEAM2.WORK;")



# Create or replace a table named "noaa_gsod" with specific columns and data types
# The table is clustered by "station_id" and "date" for performance optimization
# The data is sourced from another table "ENVIRONMENT_DATA_ATLAS.ENVIRONMENT.NOAACD2019R" and filtered and pivoted accordingly
cs.execute(
    """
    CREATE OR REPLACE TABLE noaa_gsod
    CLUSTER BY (station_id, date) AS (
    select *
    from (
        select "Stations", "Date", "Stations Name", "Stations Latitude", "Stations Longitude", "Country", "Indicator Name", "Value"
        from ENVIRONMENT_DATA_ATLAS.ENVIRONMENT.NOAACD2019R
        where "Date">='2020-01-01'
        and "Measure"='M1'
        and "Country"='US'
    )
    pivot(max("Value") for "Indicator Name" in ('Mean visibility (miles)','Maximum temperature (Fahrenheit)','Mean dew point (Fahrenheit)','Maximum wind gust (Number)','Minimum temperature (Fahrenheit)','Maximum sustained wind speed (knots)','Mean wind speed (knots)','Mean station pressure (millibars)','Precipitation amount (inches)','Mean temperature (Fahrenheit)','Mean sea level pressure (millibars)','Snow depth (inches)')
    ) as p(station_id, date, name, latitude, longitude, country_fips, visibility, max, dew, wind_max, min, wind_sustained_max, wind_mean, pressure, rain, temp, pressure_sea, snow_depth)
    );

    """
)

In [30]:
cs.execute("USE PROJECT_TEAM2.WORK;")

# Create or replace a stage named "latlon_stg" with CSV file format
cs.execute("CREATE OR REPLACE STAGE latlon_stg FILE_FORMAT = (TYPE = 'CSV')")

# Define the Snowflake stage to upload files to
stage = 'PROJECT_TEAM2.WORK.latlon_stg'


# Function to guess the data type of a value
def guess_data_type(value):
    try:
        int(value)
        return 'INT'
    except ValueError:
        try:
            float(value)
            return 'FLOAT'
        except ValueError:
            return 'STRING'

# Function to generate SQL for creating a table based on the first row of a CSV file
def generate_create_table_sql(csv_path, table_name):
    with open(csv_path, 'r') as f:
        reader = csv.reader(f)
        headers = next(reader)
        first_row = next(reader)

    column_types = [guess_data_type(value) for value in first_row]
    column_definition = ", ".join(f"{header} {data_type}" for header, data_type in zip(headers, column_types))

    create_table_sql = cs.execute(f"CREATE OR REPLACE TABLE {table_name} ({column_definition})")
    return create_table_sql


# Define the path to the CSV file and the table name
csv_path = './simplemaps_uszips_basicv1.82/uszips.csv'
table_name = 'LATLONGINFO'

# Generate the SQL for creating the table and execute it
create_table_sql = generate_create_table_sql(csv_path, table_name)

# Upload the CSV file to the Snowflake stage
cs.execute(f"PUT file://./simplemaps_uszips_basicv1.82/uszips.csv @{stage}")

# Copy data from stage to table
cs.execute(f"COPY INTO LATLONGINFO FROM @{stage} FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER = 1) ON_ERROR = 'CONTINUE'")

In [34]:
cs.execute("USE PROJECT_TEAM2.WORK;")

# Create or replace a view named "supplier_zip_code_weather"
# The view joins the "SUPPLIER_CASE", "LATLONGINFO", and "NOAA_GSOD" tables
# It calculates the closest weather station to each supplier's ZIP code and selects the maximum temperature
# The view is based on a CTE that ranks the weather stations by distance for each ZIP code
cs.execute("""
CREATE OR REPLACE VIEW supplier_zip_code_weather AS (
    WITH ranked_data AS (
        SELECT A.POSTALPOSTALCODE, C.DATE, C.MAX, 
               st_distance(st_makepoint(B.LNG, B.LAT), st_makepoint(C.LONGITUDE, C.LATITUDE)) AS distance,
               row_number() OVER (PARTITION BY A.POSTALPOSTALCODE ORDER BY st_distance(st_makepoint(B.LNG, B.LAT), st_makepoint(C.LONGITUDE, C.LATITUDE))) as rank
        FROM SUPPLIER_CASE A
        INNER JOIN LATLONGINFO B ON A.POSTALPOSTALCODE = B.ZIP
        INNER JOIN NOAA_GSOD C ON st_distance(st_makepoint(B.LNG, B.LAT), st_makepoint(C.LONGITUDE, C.LATITUDE)) < 50000
    )
    SELECT POSTALPOSTALCODE, DATE, MAX(MAX) AS MAX_TEMP
    FROM ranked_data
    WHERE rank = 1
    GROUP BY POSTALPOSTALCODE, DATE
    ORDER BY POSTALPOSTALCODE, DATE
);
""")

Exercise 8

In [115]:
# Create or replace a view named "combined_orders_and_weather"
# The view joins the "purchase_orders_and_invoices", "supplier_zip_code_weather", and "supplier_case" tables
# It selects various columns from these tables, including supplier ID, supplier name, purchase order ID, order date, and maximum temperature
# The view filters out records where the maximum temperature is null



cs.execute("""
CREATE OR REPLACE VIEW combined_orders_and_weather AS (
    SELECT C.supplierid, C.suppliername, A.purchaseorderid, A.orderdate, B.postalpostalcode, B.MAX_TEMP
    FROM purchase_orders_and_invoices A
    INNER JOIN supplier_zip_code_weather B ON A.Orderdate = B.Date
    INNER JOIN supplier_case C ON B.POSTALPOSTALCODE = C.POSTALPOSTALCODE
    WHERE B.MAX_TEMP IS NOT NULL
);
""")